In [5]:
import pandas as pd

df = pd.read_csv("./data/gemstone.csv")
df.head()


,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [7]:
df = df.drop(labels=["id"],axis=1)

In [9]:
#indpendent and dependent features
X = df.drop(labels="price",axis=1)
y = df[["price"]]

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [10]:
#define clomuns 
catg_cols = X.select_dtypes(include="object").columns
nume_cols = X.select_dtypes(exclude="object").columns

In [14]:
## Define the custom ranking for each ordinal variable
    # df["cut"].unique()
    # df["cut"].nunique()
    # df["cut"].value_counts()

cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [21]:
from sklearn.impute import SimpleImputer #handling missing values
from sklearn.preprocessing import StandardScaler #handling feature scaling
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


#Numarical Pipeline
nume_pipeline =Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="median")),
        ("scaler",StandardScaler())
    ]
)


catg_pipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy='most_frequent')),
        ("ordinalencoder",OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ("scaler",StandardScaler())    
    ]
)

preprocessor = ColumnTransformer([
    ("nume_pipeline",nume_pipeline,nume_cols),
    ("catg_pipeline",catg_pipeline,catg_cols)
])


In [38]:
#Train_test and model_training
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import numpy as np


X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.30,random_state=42)

X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())


def evaluate_model(true,predicated):
    mae = mean_absolute_error(true,predicated),
    mse = mean_squared_error(true,predicated),
    rmse=np.sqrt(mse)
    r2_square=r2_score(true,predicated)
    return mae,mse,rmse,r2_square

models={
    'LinearRegrssion':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
model_list =[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    #make predictions
    y_pred = model.predict(X_test)
    mae,mse,rmse,r2_square=evaluate_model(y_test,y_pred)
    
    model_list.append(list(models.keys())[i])
    print(list(models.keys())[i],'Model Training Performance')
    print("RMSE:",rmse)
    print("MsE:",mse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)
    
    print('*'*30)
    print('\n')

    

LinearRegrssion Model Training Performance
RMSE: [1014.62966304]
MsE: (1029473.3531156846,)
MAE: (675.0758270067483,)
R2 score 93.62906819996049
******************************


Lasso Model Training Performance
RMSE: [1014.65913028]
MsE: (1029533.1506505491,)
MAE: (676.2421173665509,)
R2 score 93.62869814082755
******************************


Ridge Model Training Performance
RMSE: [1014.63432335]
MsE: (1029482.8101268951,)
MAE: (675.1077629781341,)
R2 score 93.62900967491632
******************************


ElasticNet Model Training Performance
RMSE: [1533.35412459]
MsE: (2351174.871397875,)
MAE: (1060.9432977143008,)
R2 score 85.44967219374031
******************************




In [37]:
model_list

['LinearRegrssion', 'Lasso', 'Ridge', 'ElasticNet']